In [28]:
import asf_search as asf
import numpy as np
import geopandas as gpd
import pandas as pd
# import holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')
import datetime
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import GeoJSONDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure,show
import json

In [2]:
def make_df(results):
    df = pd.DataFrame(
    {
        'fileID':[],
        'platform':[],
        'flightDirection':[],
        'centerLat':[],
        'centerLon':[],
#         'processingLevel':[],
        'startTime':[]
    })
    name = []
    platformlist = []
    flightlist = []
    latlist = []
    lonlist = []
#     processlist = []
    stime = []
    for i in range(len(results)):
        name.append(results[i].properties['fileID'])
        platformlist.append(results[i].properties['platform'])
        flightlist.append(results[i].properties['flightDirection'])
        latlist.append(float(results[i].properties['centerLat']))
        lonlist.append(float(results[i].properties['centerLon']))
#         processlist.append(results[i].properties['processingLevel'])
        timestring = results[i].properties['startTime'][0:10] + ' ' + results[i].properties['startTime'][11:]
        stime.append(timestring)

    df.fileID = name
    df.platform = platformlist
    df.flightDirection = flightlist
    df.centerLat = latlist
    df.centerLon = lonlist
#     df.processingLevel = processlist
    df.startTime = stime
    
    return df

In [40]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 100,
    'start': '2016-01-01T00:00:00Z',
    'end': '2016-12-31T23:59:59Z'
}
results = asf.search(**opts)

df_2016 = make_df(results)
gdf = gpd.GeoDataFrame(df_2016, geometry=gpd.points_from_xy(df_2016.centerLon, df_2016.centerLat))

In [83]:
type(results)

asf_search.ASFSearchResults.ASFSearchResults

In [68]:
gdf.drop(['geometry'], axis=1,inplace=True)

In [6]:
gdf.to_csv('Sentinel1_2016.csv')

In [86]:
def get_dataset(src):
    gdf = src.copy()
    gdf['date'] = gdf['startTime']
    gdf = gdf.set_index(['date'])
    gdf.sort_index(inplace=True)
    gdf = gdf.to_dict()

    return GeoJSONDataSource(geojson=json.dumps(gdf))

In [16]:
def make_plot(source, title):
    #     plot = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
    plot = figure()
    plot.title.text = title

    plot.line('date','centerLat',source=source)

    # fixed attributes
    #     plot.xaxis.axis_label = None
    #     plot.yaxis.axis_label = "Latitude"
    #     plot.axis.axis_label_text_font_style = "bold"
    #     plot.x_range = DataRange1d(range_padding=0.0)
    #     plot.grid.grid_line_alpha = 0.3

    return plot

In [87]:
source = get_dataset(gdf)

In [88]:
plot = make_plot(source, "Ground-swath project")

In [89]:
curdoc().add_root(row(plot))
curdoc().title = "Ground-swath project"

inline